---
## Комментарий от наставника

Привет! Спасибо, что сдал задание:) В целом, работа производит положительное впечатление, однако обрати внимание на некоторые мои комментарии. Они обозначены пометкой *Комментарий от наставника*. Пострайся учесть эти комментарии в ходе дальнейшей работы.

---

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

---
## Комментарий от наставника


Правильно, что есть краткое вступление в работу, описание того, что надо делать. В работе необходимо приводить краткий план того, что надо сделать(если этот план имеется), а также информацию о входных данных: какие столбцы есть в таблице, их названия и какую информацию они несут (см. пример). Также название работы лучше отображать в отдельной ячейке и делать крупный шрифт (заголовок). Так работа выглядит презентабельно.

<font color=red> Пример: </font>
---
Описание данных: 
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- ...
    
---

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Немного <b>переименуем столбцы</b> для удобства понимания их значений:<br/>
-'debt' -> 'credit_fail' (имел ли задолженность по возврату кредитов);<br/>
-'income_type' -> 'type' (тип занятости);<br/>
-'total_income' -> 'salary' (ежемесячный доход)

In [6]:
data = data.rename(columns={'debt':'credit_fail', 'income_type':'type', 'total_income':'salary'})
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<b><font size="+1">1.1. </b></font>Получим <b>общую информацию</b>:

In [7]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,credit_fail,salary
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<b>Наблюдения 1:</b><br/>
1. Странное минимальное значение по <b>'children'</b> - -1. Вероятно, это просто отсутствующая информация (хотя это ведь кредиты).<br/>
2. Максимальное значение по <b>'children'</b> - 20. Выглядит реалистично, но проверим, сколько таких строк имеется в датафрейме. Вполне возможно, что это ошибка.<br/>
2. Отрицательные значения по количеству отработанных дней <b>'days_employed'</b> - вполне возможно, что если возьмем значения по модулю, то данные станут пригодными для работы.<br/>
3. Среднее значение по количеству отработанных дней <b>'days_employed'</b> - 173 года (63046 дней). Кто-то нам нарушает корректность картины.<br/>
4. Возраст по некоторым строкам <b>'dob_years'</b> равен 0 - предполагаю, что просто не указан (хотя, опять же, это ведь кредиты!).

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
type                21525 non-null object
credit_fail         21525 non-null int64
salary              19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<b>Наблюдения 2:</b><br/>
1. Отсутствуют значения только в столбцах <b>'days_employed'</b> и <b>'salary'</b>.<br/>
2. Столбцам <b>'days_employed'</b> и <b>'salary'</b> стоит присвоить целочисленные значения (текущая точность нам не нужна)

---
## Комментарий от наставника

Хорошо бы выводить больше 10 строчек таблицы. Меньше не рекомендуется, можно не увидеть структуру данных. У меня есть несколько комментариев:
- желательно давать переменным осознанные имена. Таблица про клиентов - название следовало выбрать соответствующее;
- выводы грамотные. Помимо результатов желательно высказывать и гипотезы о том, как исправить неточности или ошибки в данных. 

Подумай также о возможных причинах их появления, а также о том, являются ли они случайными или нет. Все свои соображения о выявленных закономерностях необходимо писать.

---


    
<b><font size="+1">1.2. </b></font><b>Проверим</b>, это одни и те же строки, где отсутствуют данные по обоим столбцам (<b>'days_employed'</b> и <b>'salary'</b>) или нет.

In [9]:
data[(data['salary'].isnull() == True) & (data['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
type                2174 non-null object
credit_fail         2174 non-null int64
salary              0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


<b>Предположение подтвердилось</b> - в строках, где отсутствуют данные в столбце <b>'days_employed'</b>, отсутствуют данные и по <b>'salary'</b>.

Проверим, из разных ли это профессий люди (если из одной, то можно будет заполнить NaN средним по этой группе).

In [10]:
data[(data['salary'].isnull() == True) & (data['days_employed'].isnull() == True)]['type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: type, dtype: int64

Из разных. Значит, взять среднее по одной группе профессий (<b>'type'</b>) уже нельзя, будем заполнять NaN по среднему из каждой группы.


<b><font size="+1">1.3. </b></font>Теперь разберемся с корректностью данных.

Выведем некоторые данные по столбцу <b>'days_employed'</b> (чтоб разобраться с теми 2 странностями):

In [11]:
print("Количество строк с 'days_employed' > 0:", data[data['days_employed'] > 0].shape[0])
print("Количество строк с 'days_employed' < 0:", data[data['days_employed'] < 0].shape[0])
data_pens = data[(data['days_employed'] > 0) & (data['type'] == 'пенсионер')]
print("Количество пенсионеров с 'days_employed' > 0:", data_pens.shape[0])
print("Количество пенсионеров с 'days_employed' > 0 и кол-вом отработанных лет > 20:", data_pens[data_pens['days_employed'] > 33000].shape[0])
print("Среднее количество отработанных дней среди пенсионеров:", data_pens['days_employed'].mean())

Количество строк с 'days_employed' > 0: 3445
Количество строк с 'days_employed' < 0: 15906
Количество пенсионеров с 'days_employed' > 0: 3443
Количество пенсионеров с 'days_employed' > 0 и кол-вом отработанных лет > 20: 3443
Среднее количество отработанных дней среди пенсионеров: 365003.4912448612


Выведем некоторые данные по столбцам <b>'dob_years'</b> и <b>'children'</b> (чтоб разобраться с теми 2 странностями):

In [12]:
print("Количество людей с нулевым возрастом:", data[data['dob_years'] == 0].count()[0])
print("Количество людей с -1 ребенком:", data[data['children'] == -1].count()[0])
print("Количество строк с 20 детьми:", data[data['children'] == 20].count()[0])
print("Количество уникальных людей с 20 детьми:", len(data[data['children'] == 20]['salary'].unique()))

Количество людей с нулевым возрастом: 101
Количество людей с -1 ребенком: 47
Количество строк с 20 детьми: 76
Количество уникальных людей с 20 детьми: 68


<b>Мы получили, что: </b><br/>
1) ~75% значений из 'days_employed' отрицательные и не относятся к пенсионерам - просто возьмем весь столбец по модулю, чтоб убрать это недоразумение;<br/>
2) остальные практически все значения > 0 (кроме двух) - это пенсионеры, причем по всем из них стаж работы > хоть немного разумных 90 лет, что, при всем уважении, выглядит абсурдом;<br/>
3) нулевой возраст для некоторых клиентов - предполагаю,что данные просто были не внесены в базу (исключаю их отсутствие, т.к. это всё-таки процедура выдачи кредита). Надо исправлять;<br/>
4) "-1 ребенок" - к исправлению;<br/>
5) "20 детей" - это не уникальный случай, а просто ошибка (может быть, корректное значение - "2"). Тоже к исправлению.

---
## Комментарий от наставника

Все верно. Грамотные выводы по всем обнаруженным в данных ошибкам.

---

### Вывод (над чем работаем):

<html>
<head>
<title>HTML код таблицы, примеры</title>
</head>
<body>
<table border="1" table-layout: fixed>
<tr>
<td style="background-color:#ffffcc">Обработка пропусков (и странных значений)</td>
<td style="background-color:#ffffcc">Замена типа данных</td>
<td style="background-color:#ffffcc">Обработка дубликатов</td>
<td style="background-color:#ffffcc">Лемматизация</td>
<td style="background-color:#ffffcc">Категоризация данных</td>
</tr>
<tr>
<td>1. Попарные NaN <b>'days_employed'</b> и <b>'salary'</b> заполним значением по данной группе <b>'type'</b> в соотношении с возрастом. </td>
<td>1. Столбцам <b>'days_employed'</b> и <b>'salary'</b> стоит присвоить целочисленные значения (текущая точность нам не нужна) </td>
<td>1. Находим дубликаты, просматриваем их и обрабатываем. </td>
<td>1. В столбце <b>'purpose'</b> есть похожие друг на друга записи, но выраженные разными конструкциями, - будем проводить лемматизацию для вычисления уникальных видов целей, на которые брали кредиты:<br/>. </td>
<td>1. На основании полученного после лемматизации листа уникальных и топовых основ слов из столбца <b> 'purpose'</b> создаем новый столбец с категориями (численными).
</td>
</tr>
<tr>
<td>2. Обрабатываем отрицательные значения по количеству отработанных дней <b>'days_employed'</b> (проверяем, возможно ли взять значение по модулю).</td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td>1) сначала найдем уникальные слова в начальных формах (использование модуля <b>pymystem3</b>). </td>
<td>2. Проверим уникальность значений по столбцам и правильность уже выполненой категоризации по следующим столбцам:<br/><br/>
    -<b>'education'-'education_id'</b>;<br/>
    -<b>'family_status'-'family_status_id'</b>;<br/><br/>
Предварительно они выглядят благородно, но проверка необходима.
</td>
</tr>
<tr>
<td>3. Столбец <b>'education'</b> имеет не однотипные значения (напр., 'Среднее'-'среднее'-'СРЕДНЕЕ') - необходимо будет привести к общему виду. </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td>2) потом выберем некий топ (без глаголов, пробелов и предлогов) и возьмем их основы при помощи модуля <b>nltk.stem</b></td>
<td>3. <b>'days_employed'</b> (после обработки пенсионеров) разбиваем на 3 группы: <br/>
    -до 10 лет;<br/>
    -10-30 лет;<br/>
    -более 30 лет.</td>
</tr>
<tr>
<td>4. Исправляем минимальное значение по <b>'children'</b> (-1) на 0. </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td> 4. Разбиваем всех заемщиков по количеству детей <b>'children'</b>:<br/>
    -без детей;<br/>
    -1-2 ребенка;<br/>
    -многодетные.<br/>
</td>
</tr>
<tr>
<td>5. Обрабатываем нулевой возраст по некоторым строкам <b>'dob_years'</b>.<br/> </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td> 5. по возрасту (<b>'dob_years'</b>):<br/>
-'до 30 лет';<br/>
-'30-45 лет';<br/>
-'45-65 лет';<br/>
-'более 65 лет'. </td>
</tr>
<tr>
<td><b>6. 'days_employed'</b> по пенсионерам > 90 лет - надо приводить к нормальным значениям. <br/> </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
</tr>
<tr>
<td><b>7. 'children'</b> > 20 лет - надо приводить к нормальным значениям. <br/> </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
<td> &nbsp; </td>
</tr>
</table>
</body>
</html>

### Шаг 2. Предобработка данных

### 2.1. Обработка пропусков (пп. 1-2).

<b>2.1.1. </b> Сделаем pivot_table по каждому из типов 'type' для того, чтоб разобраться с отрицательным стажем и выяснить, кто создает перекос значения среднего стажа, которое равно 1000 годам.<br/><br/> Для каждого типа 'type' выведем следующее:<br/>
1) общее кол-во строк данного типа в таблице;<br/>
2) среднее значение по <b>'days_employed'</b>;<br/>
3) для проверки корректности вычисления среднего и применения abs() выведем кол-во значений по <b>'days_employed'</b> > 0 по данному 'type'.<br/>

In [13]:
data_grouped = data.groupby('type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})

dict_to_rename = dict(zip(data_grouped.columns.levels[1], ['Общее количество строк', 'Среднее', 'Кол-во значений > 0']))

data_grouped = data_grouped.rename(columns=dict_to_rename, level=1)
data_grouped

days_employed                                   
                Общее количество строк        Среднее Кол-во значений > 0
type                                                                     
безработный                          2  366413.652744                 2.0
в декрете                            1   -3296.759962                 0.0
госслужащий                       1312   -3399.896902                 0.0
компаньон                         4577   -2111.524398                 0.0
пенсионер                         3443  365003.491245              3443.0
предприниматель                      1    -520.848083                 0.0
сотрудник                        10014   -2326.499216                 0.0
студент                              1    -578.751554                 0.0

<b> Что имеем:</b><br/> значения по отработанным дням ('days_employed') однотипны: по всем "безработным" и "пенсионерам" все значения > 0, по остальным - < 0, поэтому можем безболезненно применять abs(), считать среднее.

<b>2.1.2. </b>Делаем следующее: 
1. Берем значения по <b>'days_employed'</b> и <b>'salary'</b> по модулю.<br/>
2. Добавляем столбец <b>'ratio_days_employed'</b> для вычисления коэффициента полноты количества отработанных дней (учитывая то, что официально можно трудиться с 16 лет), чтобы впоследствии заполнить NaN по <b>'days_employed'</b> с учетом среднего этого показателя по группе<br/>
<font size="-1">(избегаем столбцов, где в столбце <b>'days_employed'</b> NaN через notnull())</font>

In [14]:
data[['salary', 'days_employed']] = data[['salary', 'days_employed']].abs()
data['ratio_days_employed'] = data[data['days_employed'].notnull()]['days_employed']/((data['dob_years']-16)*365)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


<b>2.1.3. </b> Заполняем пропуски NaN в датафрейме data:<br/>
-<b>'days_employed'</b>: NaN заменяем на произведение среднего коэффициента по группе <b>'type'</b>, умноженного на возраст в днях;<br/>
-<b>'salary'</b>: NaN заменяем на среднее значение по зарплате по группе <b>'type'</b>.

In [15]:
data['days_employed'] = data.groupby('type')['days_employed'].transform(lambda x: x.fillna(x.mean()*data['dob_years']*365))
data['salary'] = data.groupby('type')['salary'].transform(lambda x: x.fillna(x.mean()))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


<b>2.1.4. </b> </font> Удаляем вспомогательный столбец и выводим общую информацию по датафрейму.

In [16]:
data = data.drop('ratio_days_employed', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
type                21525 non-null object
credit_fail         21525 non-null int64
salary              21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


---
## Комментарий от наставника

Грамотная обработка пропусков, молодец. Теперь пенсионеры получают, как другие пенсионеры и т.д. Логика подобных замен хорошо прослеживается.

---

### 2.2. Обработка странных значений (пп. 3-6)

<b>К работе:</b><br/>
-<b>'days_employed'</b> по пенсионерам > 90 лет - надо что-то делать. Думаю, сделать следующее: пусть значения по стажу (<b>'days_employed'</b>) у пенсионеров некорретны, всё равно, объективно, их стаж будет в среднем больше, чем у остальных групп. Поэтому вспоследствии просто присвоим каждому пользователю признак по стажу. И все пенсионеры со стажем 60 лет и 6000 лет будут отнесены к одной категории. <br/>
-столбец <b>'education'</b> имеет неоднотипные значения (напр., 'Среднее'-'среднее'-'СРЕДНЕЕ') - необходимо будет привести к общему виду;<br/>
-исправляем минимальное значение по <b>'children'</b> (-1) на 0;<br/>
-обрабатываем нулевой возраст по некоторым строкам <b>'dob_years'</b>.

In [17]:
# 'education' в low-индекс
data['education'] = data['education'].str.lower()

In [18]:
# 'children' (-1) на 0
print("Количество строк с 'children' -1 -", data[data['children'] == -1].count()[0])
print("Количество строк с 'children' 0 -", data[data['children'] == 0].count()[0])
print("Количество строк с 'children' 20 -", data[data['children'] == 20].count()[0])

Количество строк с 'children' -1 - 47
Количество строк с 'children' 0 - 14149
Количество строк с 'children' 20 - 76


1. Всего 47 строк со значением по количеству <b>детей равному "-1", изменим на "0"</b>. Статистику не испортит, а выглядеть будет по-человечески.<br/>
2. Те, у кого <b>20 детей</b>, впоследствии <b>отнесен к многодетным</b> (>2 детей).

In [19]:
data['children'] = data['children'].replace(-1, 0)

In [20]:
# смотрим количество нулей по возрасту (столбец 'dob_years') и количество несовершеннолетних
print("Количество строк с 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])
print("Количество строк с 'dob_years' < 19 -", data[data['dob_years'] < 19].count()[0])


Количество строк с 'dob_years' 0 - 101
Количество строк с 'dob_years' < 19 - 101


Количество одних и других совпадает => это одни и те же люди. Заполним значения (округляя, чтоб сохранить 'int' по этому столбцу) по ним средним по каждой группе

In [21]:
data['dob_years'] = data.groupby('type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))

Проверяем количество оставшихся строк с нулевым значением по <b>'dob_years'</b>:

In [22]:
print("Количество строк с 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])

Количество строк с 'dob_years' 0 - 0


In [23]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,credit_fail,salary
count,21525.000000,2.152500e+04,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.541092,1.522834e+08,43.495238,0.817236,0.972544,0.080883,1.673959e+05
std,1.379943,1.078074e+09,12.230322,0.548138,1.420324,0.272661,9.790695e+04
min,0.000000,0.000000e+00,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1.024652e+03,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2.605748e+03,43.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,3.336411e+05,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,20.000000,9.725518e+09,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Все NaN по датафрейму заполнены, явно некорректные значения откорректированы.

### 2.2. Замена типа данных

Месячную зарплату(<b>'salary'</b>) и стаж в днях (<b>'days_employed'</b>) переводим в <b>'int'</b>.<br/>
Используем функцию <b>'astype'</b>, т.к. столбцы у нас имеют чистый формат <b>'float64'</b>, т.е. перевод в <b>'int'</b> будет безопасным.

In [24]:
data['days_employed'] = data['days_employed'].astype('int')
data['salary'] = data['salary'].astype('int')

In [25]:
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
type                object
credit_fail          int64
salary               int64
purpose             object
dtype: object

### Вывод

Все данные представлены в довольно удобном формате. Месячную зарплату и стаж в днях перевели в <b>'int'</b>, чтобы видеть целочисленные значения. В остальном типа данных везде выглядят хорошо.

---
## Комментарий от наставника

Исходя из общей информации о таблице, мы видим, что столбец доходов имеет вещественный тип данных, который и нужно было заменить на целочисленный. Замена выполнена верно. Также хорошо научиться использовать метод to_numeric().

---

### 2.3. Обработка дубликатов

Ещё раз посчитаем количество дубликатов.<br/>
Используем стандартный метод "<b>duplicated()</b>", выводим суммарное количество дубликатов по датафрейму:

In [26]:
data.duplicated().sum()

71

Посмотрим на них:

In [27]:
data[data.duplicated(keep=False)].sort_values(by=['salary', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose
17787,0,7194218812,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
21415,0,7194218812,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
3344,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
9627,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
13300,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0,34681788,45,среднее,1,гражданский брак,1,F,компаньон,0,202417,свадьба
9920,0,39306026,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
15991,0,39306026,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
2254,0,41618145,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью


И правда выглядят как дубликаты. <b>Причина?</b> Предполагаю, что банальное задвоение данных/ошибочное создание дублей.<br/>
Удаляем их.

In [28]:
data = data.drop_duplicates()

Проверим ещё раз

In [29]:
data.duplicated().sum()

0

### Вывод

Лишили датафрейм и дубликатов.

---
## Комментарий от наставника

Дубликаты вызывают смещение наших финальных оценок. Именно поэтому их и надо удалять. Метод удаления дубликатов использован верно.

---

### 2.4. Лемматизация

В датафрейме есть столбец (<b>'purpose'</b>), рассказывающий о целях получения кредита. В нем содержатся похожие друг на друга цели, но выраженные разными словами. Необходимо выяснить количество уникальных лемм и добавить ещё 1 столбец (<b>'purpose_cat'</b>) с категорией цели кредита.

<b><font size="+1">2.4.1. </b></font> Сначала выведем список уникальных целей (<b>'purpose'</b>)

In [30]:
unique_purposes = data['purpose'].value_counts().index.tolist()
print(data['purpose'].value_counts().to_frame())

                                        purpose
свадьба                                     791
на проведение свадьбы                       768
сыграть свадьбу                             765
операции с недвижимостью                    675
покупка коммерческой недвижимости           661
операции с жильем                           652
покупка жилья для сдачи                     651
операции с коммерческой недвижимостью       650
жилье                                       646
покупка жилья                               646
покупка жилья для семьи                     638
строительство собственной недвижимости      635
недвижимость                                633
операции со своей недвижимостью             627
строительство жилой недвижимости            624
покупка недвижимости                        621
покупка своего жилья                        620
строительство недвижимости                  619
ремонт жилью                                607
покупка жилой недвижимости              

<b><font size="+1">2.4.2. </b></font> Воспользуемся модулем <b>pymystem3</b> и лемматизируем полученный выше список, получив "рейтинг" уникальных лемм:

In [31]:
m = Mystem()
list_of_lemmas = []
for element in data['purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

<b><font size="+1">2.4.3. </b></font> Считаю, что необходимо взять топ-8 без пробелов и брать слова длиной 4 и более букв - так мы оставим только существительные (они самые популярные) и лишимся пробелов и предлогов.<br/>
Образуем отдельный список <b>final_list_of_purposes</b> с нашим топ-8 целей для получения кредита.

In [32]:
final_list_of_purposes = [k for k in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                          if len(k) > 4 if k != ' ' if k != '\n'][0:8]
print(final_list_of_purposes)

['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство']


<b>Чистим список:</b><br/>
-<b>уберем "покупка"</b>, т.к. это слишком общее понятие;<br/>
-<b>уберем "строительство"</b>, т.к. это относится к жилью (судя по выгрузке выше в 2.4.1);<br/>
-<b>уберем "операция"</b>, т.к. это тоже относится к жилью (судя по выгрузке выше в 2.4.1).

In [33]:
final_list_of_purposes.remove('покупка') 
final_list_of_purposes.remove('строительство')
final_list_of_purposes.remove('операция')
print(final_list_of_purposes)

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']


---
## Комментарий от наставника

Логика верна. Хорошо бы на данном этапе подробно объяснить данный выбор. Метод lemmatize() использован правильно. Желательно подробно описывать все свои действия.

-----

Но нам нужно получить основы этих слов (для корректного присвоения категории):

In [34]:
russian_stemmer = SnowballStemmer('russian')

stemmed_purposes = [russian_stemmer.stem(word) for word in final_list_of_purposes]
print(stemmed_purposes)

['недвижим', 'жил', 'автомобил', 'образован', 'свадьб']


Преобразуем лист в словарь, чтоб присваивать строкам категории:<br/>
1 - приобретение недвижимости/жилья (присвоим этим двум словам одну и ту же категорию);<br/>
2 - приобретение автомобиля;<br/>
3 - образование;<br/>
4 - свадьба.

---
## Комментарий от наставника

Это делать необязательно. Стемминг и лемматизация - это два похожих метода обработки текстовой информации. Они взаимозаменяемы.

-----

In [35]:
dict_stemmed_purposes = {v:k for k,v in enumerate(stemmed_purposes)}
dict_stemmed_purposes['недвижим'] = 1
print(dict_stemmed_purposes)

{'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}


### Вывод

Мы получили <b>список целей</b> получения кредита в виде <b>словаря {цель:номер типа}</b>. <br/>
На следующем этапе, помимо остального, создадим новый столбец и дадим каждой строке категорию из этого списка и проверим, все ли строки оказались "окатегоризированы".

### 2.5. Категоризация данных

<b><font size="+1">2.5.1. </b></font> Начнем с создания нового столбца <b>'purpose_cat'</b> с категориями по целям получения кредита <b>'purpose'</b> на основе созданного нами выше словаря <b>dict_stemmed_purposes</b>:

In [36]:
data['purpose_cat'] = 0
for row in range(len(data)):
    for purpose in dict_stemmed_purposes:
        if purpose in data.iloc[row, 11]:
            data.iloc[row, 12] = dict_stemmed_purposes[purpose]
        
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose,purpose_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1


---
## Комментарий от наставника

Категоризация по цели кредита выполнена верно. Однако можно обойтись и без стемминга.

-----

<b><font size="+1">2.5.2. </b></font> Проверяем, не осталось ли нулей в новом столбце 

In [37]:
data[data['purpose_cat'] == 0].count().sum()

0

<b><font size="+1">2.5.3. </b></font> Необходимо проверить корректность всех пар <b>'education'</b>-<b>'education_id'</b> и <b>'family_status'</b>-<b>'family_status_id'</b>, то есть те данные, которым уже присвоены категории

In [38]:
data.groupby(['education', 'education_id']).size().to_frame('count').reset_index()

,education,education_id,count
0,высшее,0,5250
1,начальное,3,282
2,неоконченное высшее,2,744
3,среднее,1,15172
4,ученая степень,4,6


In [39]:
data.groupby(['family_status', 'family_status_id']).size().to_frame('count').reset_index()

,family_status,family_status_id,count
0,Не женат / не замужем,4,2810
1,в разводе,3,1195
2,вдовец / вдова,2,959
3,гражданский брак,1,4151
4,женат / замужем,0,12339


<b>Итог: </b>пары уникальны.

---
## Комментарий от наставника

Почему разбиение по доходу именно на такие категории? Все свои действия необходимо объяснять.

-----

<b><font size="+1">2.5.4. </b></font> Разобьем на классы по <b>'salary'</b> <br/>
-<b>'Бедный'</b> (<= 50000р.);<br/>
-<b>'Средний'</b> (50000 < x <= 120000);<br/>
-<b>'Зажиточный'</b> (120000 < x < 1000000);<br/>
-<b>'Миллионер'</b> (x >= 1000000);<br/>

по стажу (<b>'days_employed'</b>):<br/>
-'до 10 лет';<br/>
-'10-30 лет';<br/>
-'более 30 лет';<br/>

по возрасту (<b>'dob_years'</b>):<br/>
-'до 30 лет';<br/>
-'30-45 лет';<br/>
-'45-65 лет';<br/>
-'более 65 лет';<br/>

по количеству детей (<b>'children'</b>):<br/>
-'нет детей' (0 детей);<br/>
-'1-2 ребенка';<br/>
-'многодетные'(>= 3 детей);<br/>

Для удобства напишем функции для этих целей:

In [40]:
def salary_cat(row):
    if row['salary'] <= 50000:
        return 'бедный'
    elif 50000 < row['salary'] <= 120000:
        return 'средний'
    elif 120000 < row['salary'] < 1000000:
        return 'зажиточный'
    else:
        return 'миллионер'

    
def days_employed_cat(row):
    if row['days_employed'] <= 3652:
        return 'стаж до 10 лет'
    elif 3652 < row['days_employed'] <= 6904:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'


def dob_years_cat(row):
    if row['dob_years'] < 30:
        return 'до 30 лет'
    elif 30 <= row['dob_years'] < 45:
        return '30-45 лет'
    elif 45 <= row['dob_years'] < 65:
        return '45-65 лет'
    else:
        return 'старше 65 лет'

    
def children_cat(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

In [41]:
data['salary_cat'] = data.apply(salary_cat, axis=1)
data['days_employed_cat'] = data.apply(days_employed_cat, axis=1)
data['dob_years_cat'] = data.apply(dob_years_cat, axis=1)
data['children_cat'] = data.apply(children_cat, axis=1)

In [42]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose,purpose_cat,salary_cat,days_employed_cat,dob_years_cat,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,зажиточный,стаж более 30 лет,30-45 лет,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,средний,стаж 10-30 лет,30-45 лет,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,зажиточный,стаж 10-30 лет,30-45 лет,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,зажиточный,стаж 10-30 лет,30-45 лет,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,зажиточный,стаж более 30 лет,45-65 лет,нет детей


### Вывод

Категоризировали датафрейм по следующим данным:<br/>
-<b>'education'</b> и <b>'family_status_id'</b> (изначально уже была категоризация по ним);<br/>
-по уровню зарплаты (<b>'salary'</b>);<br/>
-по стажу работы (<b>'days_employed'</b>);<br/>
-по количеству детей (<b>'children'</b>).<br/><br/>

Переходим к анализу полученных данных.

### Шаг 3. Ответьте на вопросы

Функция для вывода информации

In [43]:
def relation(category):
    return data.groupby(category)['credit_fail'].mean().to_frame().sort_values(by='credit_fail')

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [44]:
relation('children_cat')

,credit_fail
children_cat,
нет детей,0.075258
многодетные,0.085526
1-2 ребенка,0.093003


### Вывод

Заемщики, не имеющие детей, менее склонны к просрочке по выплатам кредита.

---
## Комментарий от наставника

Помимо относительных величин надо приводить и число людей в данных группах, а также число должников.

-----

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [45]:
relation('family_status')

,credit_fail
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


### Вывод

Граждане, которые не имеют узаконенных отношений, более склонны к просрочке по выплатам кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
relation('salary_cat')

,credit_fail
salary_cat,
бедный,0.061828
миллионер,0.080000
зажиточный,0.081459
средний,0.081570


### Вывод

Как ни странно, заемщики с уровнем дохода < 50000р. менее склонны нарушать обязательства по выплатам кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

{'недвижимость/жилье': 1, 'автомобиль': 2, 'образование': 3, 'свадьба': 4}

In [47]:
relation('purpose_cat')

,credit_fail
purpose_cat,
1,0.072334
4,0.080034
3,0.092200
2,0.093590


### Вывод

Заемщики, берущие кредит для приобретения/проведение операций с <b>жильем</b>, наиболее ответственны и менее склонны нарушать обязательства по выплатам кредита в срок.

---
## Комментарий от наставника

На этом этапе работы желательно использовать метод сводных таблиц (pivot_table) при получении результатов. В качестве дополнительного задания попробуй построить зависимости показателей на возврат кредита, используя метод сводных таблиц хотя бы для одного пункта данного шага. Применяй его в дальнейших работах. 

Выводы верные по всем пунктам.

-----

### Шаг 4. Общий вывод

Отвечая на поставленный вопрос "Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок", могу дать следующий ответ - семеное положение и количество детей влияет на факт погашения кредита в строк:<br/>
-заемщики с <b>официально оформленными отношениями</b> (или которые в прошлом были в официальном в браке) и <b>не имеющие детей</b> - самые <span style="color:green">ответственные заемщики</span>;<br/>
-заемщики, состоящие <b>в неофициальном браке или находящиеся без отношений</b>, при этом <b>имеющие 1 или 2 детей</b> - самые <span style="color:red">менее ответственные заемщики</span>.<br/><br/>

В целом, по исследованным данным можно сделать следующий вывод:
<html>
<head>
<title>HTML код таблицы, примеры</title>
</head>
<body>
<table border="1">
<tr>
<td style="background-color:#00FF7F"> Ответственный заещик </td>
<td>Не имеет детей</td>
<td>Находится/был в официальном браке</td>
<td>Уровень дохода <b><</b> 50000</td>
<td>Берет кредит на жилье</td>
</tr>
<tr>
<td style="background-color:#ffc6c4"> Менее ответственный заещик </td>
<td>Имеет 1-2 детей</td>
<td>Не в узаконенных отношениях/не в отношениях</td>
<td>Уровень дохода <b>></b> 50000</td>
<td>Берет кредит на образование/свадьбу</td>
</tr>
</table>
</body>
</html>

---
## Комментарий от наставника

Выводы описаны грамотно, подробно, объяснены представленные доводы. По каждому пункту из задания представлен вывод, что правильно. Есть некое финальное портфолио клиентов. В выводах можно приводить полученные ранее значения, правильно. В дальнейшем надо будет проверять полученные значения с помощью статистических критериев, так как разница в 1% кажется небольшой. Желательно подробно расписать все, что было сделано и какими методами. Так ты лишний раз вспомнишь, что делает та или иная функция.

---

## Комментарий от наставника

- Необходимо научиться объяснять и описывать все свои действия и ход мыслей;
- Выдвигай больше гипотез. Пытайся понять реальны они или нет и проверяй их на практике;
- Хорошо, что оставляешь комментарии в коде. Так и тебе, и другим людям понятно, что делает определенный кусок кода;
- Изучи метод сводных таблиц и применяй его в дальнейших проектах.

Ты проделал хорошую работу, молодец! Продолжай в том же духе :)

-----

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.